In [1]:
import diff_gaussian_rasterization as dgr
from diff_gaussian_rasterization import GaussianRasterizationSettings, GaussianRasterizer
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import bayes3d as b
import jax.numpy as jnp
from random import randint
import pytorch3d.transforms

In [2]:
b.setup_visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
import meshcat.geometry as mg

In [4]:
class Ellipsoid2(mg.Sphere):
    """
    An Ellipsoid is treated as a Sphere of unit radius, with an affine
    transformation applied to distort it into the ellipsoidal shape
    """
    def __init__(self, transform):
        super(Ellipsoid2, self).__init__(1.0)
        self.transform = np.array(transform)

    def intrinsic_transform(self):
        return self.transform


In [5]:
VISUALIZER = b.get_visualizer()

In [7]:
from plyfile import PlyData, PlyElement
path = "/home/nishadgothoskar/gaussian-splatting/output/93d9b62d-d/point_cloud/iteration_30000/point_cloud.ply"
plydata = PlyData.read(path)

xyz = np.stack((np.asarray(plydata.elements[0]["x"]),
                np.asarray(plydata.elements[0]["y"]),
                np.asarray(plydata.elements[0]["z"])),  axis=1)
opacities = np.asarray(plydata.elements[0]["opacity"])[..., np.newaxis]

features_dc = np.zeros((xyz.shape[0], 3, 1))
features_dc[:, 0, 0] = np.asarray(plydata.elements[0]["f_dc_0"])
features_dc[:, 1, 0] = np.asarray(plydata.elements[0]["f_dc_1"])
features_dc[:, 2, 0] = np.asarray(plydata.elements[0]["f_dc_2"])


scale_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("scale_")]
scale_names = sorted(scale_names, key = lambda x: int(x.split('_')[-1]))
scales = np.zeros((xyz.shape[0], len(scale_names)))
for idx, attr_name in enumerate(scale_names):
    scales[:, idx] = np.asarray(plydata.elements[0][attr_name])

rot_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("rot")]
rot_names = sorted(rot_names, key = lambda x: int(x.split('_')[-1]))
rots = np.zeros((xyz.shape[0], len(rot_names)))
for idx, attr_name in enumerate(rot_names):
    rots[:, idx] = np.asarray(plydata.elements[0][attr_name])

In [15]:
b.show_cloud("xyz", xyz/ 5.0)

In [10]:
xyz

array([[-5.312956  ,  4.309359  , 27.713703  ],
       [-5.312956  ,  4.309359  , 27.713703  ],
       [-5.2696996 ,  4.190924  , 27.747124  ],
       ...,
       [-5.1862426 , -0.27020842, 24.046867  ],
       [-3.6047409 , -0.74231774, 24.905943  ],
       [-5.16791   , -0.2717039 , 24.058065  ]], dtype=float32)

In [19]:
x = np.array([3.0, 1.0, 2.0])
y = np.array([0.0, 1.0, 0.0])
z = np.array([0.0, 0.0, 1.0])
rot = np.hstack([x[:, None], y[:, None], z[:, None]])
mat = np.eye(4)
mat[:3,:3] = rot
mat[:3,3] = np.array([13.0, 6.0, 5.0])
mat
print(mat)
shape = Ellipsoid2(mat)
VISUALIZER["1"].set_object(shape)

[[ 3.  0.  0. 13.]
 [ 1.  1.  0.  6.]
 [ 2.  0.  1.  5.]
 [ 0.  0.  0.  1.]]


In [16]:
b.show_cloud("1", np.zeros((10,3)))